# 벤저민 그레이엄 기초 전략

## Basic Setting

In [49]:
import pandas as pd
import requests
from lxml import html
from tqdm import tqdm

In [51]:
snap_url = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'

In [52]:
print(snap_url)

https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701


In [53]:
ratio_url = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'

In [54]:
print(ratio_url)

https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701


In [55]:
print(snap_url.format('035720'))

https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A035720&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701


In [56]:
sample_code = '005930'

In [57]:
print(ratio_url.format('005930'))

https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A005930&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701


## Get PER

In [58]:
Snap_url = snap_url.format(sample_code)

In [59]:
snap_content = requests.get(Snap_url).content

In [60]:
snap_tree = html.fromstring(snap_content)

In [61]:
per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
per = float(per)

In [62]:
per

12.64

## Get Debt Ratio

In [63]:
Ratio_url = ratio_url.format(sample_code)

In [64]:
ratio_content = requests.get(Ratio_url).content

In [65]:
html_tree = html.fromstring(ratio_content)

In [66]:
debt_ratio = html_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
debt_ratio = float(debt_ratio)

In [67]:
debt_ratio

39.9

## Get All Stock Code

In [68]:
company = pd.read_csv('company.csv')

In [69]:
code_list = company['종목코드'].dropna()

In [70]:
code_list

0       000155
1       00088K
2       010955
3       051915
4       071055
         ...  
2017    013030
2018    019550
2019    019590
2020    019570
2021    006920
Name: 종목코드, Length: 2021, dtype: object

## Create Crawling Function

In [84]:
def crawler(li):
    result = {}
    
    for code in tqdm(li):
        try:
            # set URLs
            snap_url = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
            ratio_url = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'
        
            # get company name
            name = company[company['종목코드'] == code]['회사명'].values[0]
        
            # get PER
            Snap_url = snap_url.format(code)
            snap_content = requests.get(Snap_url).content
            snap_tree = html.fromstring(snap_content)
        
            per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
            per = float(per)
        
            # get Debt Ratio
            Ratio_url = ratio_url.format(code)
            ratio_content = requests.get(Ratio_url).content
            html_tree = html.fromstring(ratio_content)
        
            debt_ratio = html_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
            debt_ratio = float(debt_ratio)
        
            result[name] = [code, per, debt_ratio]
            
        except (TypeError, IndexError, AttributeError, ValueError):
            pass
    
    # create DataFrame
    result = pd.DataFrame(result)
    
    # transposing
    result = result.transpose()
    
    # set Column names
    column_names = ['Code', 'PER', 'Debt_Ratio']
    result.columns = column_names
    
    return result

In [85]:
result_df = crawler(code_list)

100%|██████████| 2021/2021 [11:20<00:00,  2.97it/s]  


In [87]:
copy_df = result_df.copy()

# Screening

In [90]:
final_result = copy_df[(copy_df['PER'] > 0) & (copy_df['PER'] <= 10) & (copy_df['Debt_Ratio'] <= 50)]

In [92]:
final_result = final_result.sort_values(by='PER').head(30)

In [93]:
final_result

,Code,PER,Debt_Ratio
솔브레인,036830,0.28,35.0
원익,032940,0.98,45.2
유수홀딩스,000700,1.77,23.6
KPX홀딩스,092230,1.81,31.3
한솔홀딩스,004150,2.01,18.4
신풍제지,002870,2.1,2.7
넥스트아이,137940,2.24,31.0
대덕전자,008060,2.41,19.8
삼정펄프,009770,2.55,10.6
케이엠,083550,2.6,39.6
